In [ ]:
  const response = await c3Post(typeName, method, data);
 
export async function c3Post(typeName, method, data) {
  const url = `${CONFIG.APPURL}/api/8/${typeName}/${method}`;
  const cachedToken = getPrivateKey() != "EMPTY" ? getPrivateKey() : getPrivateKey(setPrivateKey(createGlobalAuthToken()));
 
  try {
    const response = await axios.post(url, data, {
      headers: {
        'Content-Type': 'application/json',
        Authorization: CONFIG.AUTH_TOKEN,
      },
    });
 
    // Return the response data. Adjust according to your needs, for example, you might want to return `response.data` directly.
    return response;
  } catch (error) {
    // Handle errors as per your existing logic
    if (error.response) {
      console.log(error.response.data);
      console.log(error.response.status);
      console.log(error.response.headers);
      throw error.response;
    } else if (error.request) {
      console.log(error.request);
      throw error.request;
    } else {
      console.log('Error', error.message);
      throw error.message;
    }
  }
  
  function signData(privateKey, data) {
    const signer = crypto.createSign('SHA512');
    signer.update(data);
    signer.end();
    const signature = signer.sign(privateKey, 'base64');
    return signature;
  }

  function getUserId() {
    try {
      const userInfo = os.userInfo();

      console.log('Current User Information:');
      console.log(`- Username: ${userInfo.username}`);
      console.log(`- Home Directory: ${userInfo.homedir}`);
      if (process.platform !== 'win32') { // UID and GID are not available on Windows
        // console.log(`- UID: ${userInfo.uid}`);
        // console.log(`- GID: ${userInfo.gid}`);
        return userInfo.uid;
      }
    } catch (error) {
      console.error('Failed to get user information:', error);
      return "";
    }
  }

  const userId = getUserId();

  const pvtKey = generateKeyPair()['privateKey'];
  // const data = "Data to sign with RSA-SHA512";
  // const signature = signData(privateKey, data);
  // {"typ":"JWT","alg":"RS512"}
  // eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzUxMiJ9.
  const generateC3KeyAuthToken = function () {
    // WARNING: The following logic is mostly copied from boot.js
    const signAlgo = "RSA-SHA512";
    const signatureText = Date.now().toString();
    const signer = crypto.createSign(signAlgo);
    signer.update(signatureText);
    // console.log(pvtKey);
    // process.exit(0);
    const signature = signer.sign(pvtKey, "base64");
    const tokenString =
      userId +
      ":" +
      Buffer.from(signatureText).toString("base64") +
      ":" +
      signature;
    const authToken = "c3key " + Buffer.from(tokenString).toString("base64");
    console.log(authToken);
    process.exit(0);
    return authToken;
  };

In [ ]:
// example ajax call 

import { ajax } from '@c3/ui/UiSdlDataRedux';

try { 
          ajax("GuruToolSources", "getToolRequestBaseUrl", { "resultId": resultId}).toPromise().then(response => {
            setToolEndpointUrl(response.response.value);
          }).catch(error => console.error('Error fetching data:', error));
        } catch (error) {
          console.error('Error fetching data:', error);
        }